In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../pairedRS')

In [6]:
import numpy as np   
import pandas as pd
import pdb
import sys
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from time import time

In [3]:
from utils import *     # contains some useful helper functions 
from models import *    # toy models
from solvers import *   # matrix completion solvers
from methods import *

(CVXPY) Oct 13 09:58:12 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [9]:
# Define the latent weights based on the factor matrices
class SamplingBias():
    def __init__(self, m, n, standardize=True):
        self.m = m
        self.n = n
        self.shape = (self.m, self.n)
        self.std = standardize
        
    def latent_weights(self, U, V, v, a, b, scale=0.5):
        r = int(len(v)/2)
        x = np.tile(np.dot(U, v[:r]), (len(U),1)).T + np.tile(np.dot(V, v[r:]), (len(U),1))
        w = np.zeros_like(x)
        w[np.where((x <= b) & (x >= a))] = 1
        w[np.where(x > b)] = scipy.stats.norm.pdf(x[np.where(x > b)], loc=b, scale=scale)/scipy.stats.norm.pdf(b, loc=b, scale=scale)
        w[np.where(x < a)] = scipy.stats.norm.pdf(x[np.where(x < a)], loc=a, scale=scale)/scipy.stats.norm.pdf(a, loc=a, scale=scale)
        return w/np.sum(w) if self.std else w

    def unif_weights(self):
        w = np.ones(self.shape)
        return w/np.sum(w) if self.std else w

    def inc_weights(self, scale=1):
        w = np.arange(1, self.m*self.n+1)**scale/(self.m*self.n)
        w = w.reshape(self.shape)
        return w/np.sum(w) if self.std else w

## Weighted model 1
For the model outline and method details, refer to Section 5.3 in the CMC Overleaf draft.
In this setting, we consider $w_1(I_i) = \frac{i}{N}$ where $N$ is the total number of entries in the matrix, and $w_2(I_i)=\frac{1}{n_{\text{test}}}$. Roughly speaking, observations are sampled with a probability proportional to the index value and test points are uniformly sampled from the missing entries.

In [10]:
# Define experiment parameters
m, n, r = 200,200,10
prop_obs = 0.5
alpha = 0.1
random_state = 0

In [11]:
# define the weights
# spatially determined weights, lower half of the indexes are more likely to
# be observed, for instance, a database with data ranked from oldest to newest
# and newer data are comprehensive. 
scale = 1.3
biasmodel = SamplingBias(m,n)
w = biasmodel.inc_weights(scale=scale)

In [12]:
RFM = RandomFactorizationModel(m ,n, r)
U, V, M = RFM.sample_noiseless(random_state)

print('Sampling with weight from the noiseless random factorization model... \n')
print('Dark spots indicating the missing entries.')
sampler = PairSampling(n1,n2)
obs = sampler.sample_observed(prop_obs, w=w, random_state=random_state)
mask_obs = sampler.sample_submask(size_obs, random_state=random_state)
test = np.ones_like(obs) - obs
train, calib = sampler.sample_train_calib(obs, calib_size=0.2*np.sum(obs), 
                                            random_state = random_state)
assert np.sum(train+calib==obs) == m*n,'Sampling error!'
vmin, vmax = np.min(M)-100, np.max(M)
plot_before_after_mask(M, obs, vmin, vmax)
print("Observation size: {}, training size: {}, calib size: {}"
      .format(np.sum(obs), np.sum(train), np.sum(calib)))

Sampling with weight from the noiseless random factorization model... 

Dark spots indicating the missing entries.


NameError: name 'RandomSampling' is not defined